**Посчитайте, сколько компаний закрылось.**

```
SELECT COUNT(id)
FROM company
Where status = 'closed'
```
**Отобразите количество привлечённых средств для новостных компаний США.**
```
SELECT funding_total 
FROM company
WHERE category_code = 'news'
AND country_code = 'USA'
ORDER BY funding_total DESC
```
**Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.**
```
SELECT SUM (price_amount)
FROM acquisition
WHERE term_code = 'cash'
AND acquired_at BETWEEN '2011-01-01' AND '2013-12-31'
```
**Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.**
```
SELECT first_name, last_name, twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'
```
**Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.**
```
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%'
```
**Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.**
```
SELECT country_code,
SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC
```
**Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.**
```
SELECT funded_at, MIN(raised_amount), MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != 0
AND MIN(raised_amount) != MAX(raised_amount)
```
**Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.**
```
SELECT * ,
CASE
WHEN invested_companies >= 100 THEN 'high_activity'
WHEN invested_companies >= 20 AND invested_companies <100 THEN 'middle_activity'
WHEN invested_companies < 20 THEN 'low_activity'
END
FROM fund
```
**Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.**
```
SELECT CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END
ORDER BY ROUND(AVG(investment_rounds))
```
**Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.**
```
SELECT country_code, 
MIN(invested_companies), 
MAX(invested_companies), 
AVG(invested_companies)
FROM fund
WHERE founded_at between '2010-01-01' AND '2012-12-31'
GROUP BY country_code
HAVING MIN(invested_companies) !=0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10
```
**Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.**
```
SELECT first_name,last_name, instituition 
FROM people as p
LEFT JOIN education as e on p.id = e.person_id
```
**Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.**
```
SELECT c.name, COUNT(DISTINCT instituition)
FROM company as c
JOIN people as p ON p.company_id = c.id
JOIN education as e ON e.person_id = p.id
GROUP BY c.name
ORDER BY COUNT(DISTINCT instituition) desc
LIMIT 5
```
**Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.**
```
SELECT DISTINCT name
FROM company
WHERE status = 'closed'
AND id IN (SELECT company_id
FROM funding_round
WHERE is_first_round = 1 AND is_last_round = 1 )
```
**Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.**
```
SELECT distinct id
FROM people
WHERE company_id IN (SELECT DISTINCT id
FROM company
WHERE status = 'closed'
AND id IN (SELECT company_id
FROM funding_round
WHERE is_first_round = 1 AND is_last_round = 1 ))
```
**Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.**
```
SELECT DISTINCT p.id , e.instituition
FROM people as p
LEFT JOIN education as e ON e.person_id = p.id
WHERE p.id IN (SELECT distinct id
FROM people
WHERE company_id IN (SELECT DISTINCT id
FROM company
WHERE status = 'closed'
AND id IN (SELECT company_id
FROM funding_round
WHERE is_first_round = 1 AND is_last_round = 1 )))
AND e.instituition IS NOT NULL
```
**Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.**
```
SELECT DISTINCT p.id , COUNT( e.instituition)
FROM people as p
LEFT JOIN education as e ON e.person_id = p.id
WHERE p.id IN (SELECT distinct id
FROM people
WHERE company_id IN (SELECT DISTINCT id
FROM company
WHERE status = 'closed'
AND id IN (SELECT company_id
FROM funding_round
WHERE is_first_round = 1 AND is_last_round = 1 )))
AND e.instituition IS NOT NULL
GROUP BY p.id
```
**Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.**
```
WITH A AS (SELECT DISTINCT p.id,
                COUNT(e.instituition) as count
FROM people AS p
LEFT JOIN education AS e ON e.person_id = p.id
WHERE p.id IN
    (SELECT DISTINCT id
     FROM people
     WHERE company_id IN
         (SELECT DISTINCT id
          FROM company
          WHERE status = 'closed'
            AND id IN
              (SELECT company_id
               FROM funding_round
               WHERE is_first_round = 1
                 AND is_last_round = 1 )))
  AND e.instituition IS NOT NULL
GROUP BY p.id)

SELECT AVg (count)
FROM A
```
**Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.**
```
SELECT avg(count)
FROM (SELECT person_id, count(instituition) as count
FROM education
WHERE person_id IN (SELECT id
FROM people
WHERE company_id = (SELECT id
FROM company
WHERE name = 'Facebook'))
GROUP BY person_id) as a
```
**Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.**
```
SELECT f.name AS name_of_fund, 
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i JOIN company AS c ON i.company_id = c.id
JOIN fund AS f ON f.id = i.fund_id
JOIN funding_round AS fr ON fr.id = i.funding_round_id
WHERE c.milestones > 6
  AND EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN 2012 AND 2013
```
**Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.**
```
SELECT name1.name,
price,
co.name,
co.funding_total as funding_total,
ROUND ((price/funding_total), 0) as round
FROM (
SELECT acquiring_company_id, c.name, price_amount as price, acquired_company_id
FROM acquisition as a JOIN company as c ON a.acquiring_company_id = c.id) as name1
JOIN company as co ON name1.acquired_company_id = co.id
WHERE funding_total > 0 and price > 0
ORDER BY price DESC, co.name
LIMIT 10
```
**Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.**
```
SELECT c.name,
EXTRACT (MONTH FROM CAST (funded_at as date))
FROM company as c JOIN funding_round as fr
ON fr.company_id = c.id
WHERE category_code = 'social'
AND funded_at BETWEEN '2010-01-01' AND '2013-12-31'
AND raised_amount > 0
```
** Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.**
```
WITH 
a as (SELECT EXTRACT (MONTH FROM CAST (funded_at AS date)) AS MONTH,
               COUNT(DISTINCT f.name) AS count_of_fund
FROM funding_round as fr
JOIN investment as i ON fr.id = i.funding_round_id 
JOIN fund as f ON f.id = i.fund_id
WHERE EXTRACT (YEAR FROM CAST (funded_at AS date)) BETWEEN '2010' AND '2013'
AND f.country_code = 'USA'
GROUP BY MONTH),

b as (SELECT EXTRACT (MONTH FROM CAST (acquired_at as date)) as month, COUNT(acquired_company_id) as count_of_acquired, SUM (price_amount) as sum_of_acquired
FROM acquisition
WHERE EXTRACT (YEAR FROM CAST (acquired_at as date)) BETWEEN '2010' AND '2013'
GROUP BY EXTRACT (MONTH FROM CAST (acquired_at as date)))

SELECT a.month, count_of_fund, count_of_acquired, sum_of_acquired
FROM a LEFT JOIN b ON a.month = b.month
```
**Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.**
```
WITH
a as (SELECT country_code, AVG (funding_total) as year_2011
FROM company
WHERE EXTRACT (YEAR FROM CAST (founded_at as date)) = 2011
GROUP BY country_code), 

b as (SELECT country_code, AVG (funding_total) as year_2012
FROM company
WHERE EXTRACT (YEAR FROM CAST (founded_at as date)) = 2012
GROUP BY country_code),

c as (SELECT country_code, AVG (funding_total) as year_2013
FROM company
WHERE EXTRACT (YEAR FROM CAST (founded_at as date)) = 2013
GROUP BY country_code)

SELECT a.country_code,
a.year_2011,
b.year_2012,
c.year_2013
FROM a 
JOIN b ON a.country_code = b.country_code
JOIN c ON a.country_code = c.country_code
ORDER BY year_2011 DESC
```